<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=439561feec482ff76d17388cb2e9a1ca82864e8584520878f106bcd30d07de7a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-26 09:33:26
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: 73.69 K (0.52%)
Current PnL: -22.86 L (-14.98%)
CY Booked + Current PnL: -8.97 L (-5.88%)
-------------------
Total profit:  1.38 L
Total loss:  -24.25 L
-------------------
Total Booked + Current PnL: 18.01 L (14.33%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.72%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.28 L (61.76%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,-0.04,-14.14,16.57,0.09,14338.0,-14245.0,86532.0,100.85,50.0,M-SC,3.70,253.0,-0.99,0.61,13.68,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.36,-9.00,9.91,0.02,15743.0,-15703.0,158856.0,142.19,70.0,M-SC,7.90,234.0,-1.00,1.11,38.18,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-0.17,6.46,12.90,20.19,23940.0,11262.0,185582.0,-16.49,49.0,X-MC,7.43,68.0,0.47,1.30,22.55,X40,ATH,INSURANCE
43,ITC,452.00,0.40,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.38,3.79,X40,NTT,FMCG
83,VOLTAS,1530.00,0.85,7.00,11.86,19.69,24333.0,13428.0,205170.0,-3.43,49.0,X-MC,3.37,78.0,0.55,1.44,14.28,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.0,3.44,-45.97,222.31,74.15,196522.0,-75205.0,88400.0,-259.08,31.0,H-SC,16.29,149.0,-0.38,0.62,13.31,XR,NTT,CHEMICALS
76,TMPV,600.0,2.71,-14.22,65.75,42.18,155900.0,-39300.0,237110.0,-23.08,32.0,X-LC,1.02,3.0,-0.25,1.66,2.71,XY24,NTT,AUTO
0,5PAISA,593.0,2.37,-34.99,73.67,12.90,104643.0,-76453.0,142043.0,109.13,58.0,H-SC,14.00,161.0,-0.73,0.99,18.11,OX40N,NTT,FINANCE
10,BANDHANBNK,400.0,1.74,-26.77,162.74,92.42,331993.0,-74557.0,204002.0,43.68,42.0,H-SC,4.80,170.0,-0.22,1.43,17.90,XY24,NTT,BANKS
15,CAMS,4762.0,1.57,3.66,20.38,24.79,53207.0,9220.0,261076.0,-1.25,55.0,X-SC,2.08,86.0,0.17,1.83,27.65,X40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.85,-27.73,97.66,42.84,45879.0,-18028.0,46978.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.22,XY24,NTT,MISC
12,BERGEPAINT,680.00,-1.17,0.21,20.89,21.14,47590.0,473.0,227812.0,-6.89,52.0,X-MC,3.99,74.0,0.01,1.59,28.41,XY24,NTT,PAINTS
44,JCHAC,2282.00,-1.15,-24.36,32.20,-0.01,31127.0,-31138.0,96667.0,19682.61,37.0,M-SC,1.63,233.0,-1.00,0.68,9.46,OX40N,NTT,AC
38,INDIAMART,4810.62,-1.06,0.31,106.08,106.72,131245.0,387.0,123723.0,-50.25,39.0,H-SC,3.40,139.0,0.00,0.87,23.77,AR,ATH,MISC
49,LAOPALA,464.00,-0.92,-34.65,112.83,39.09,96670.0,-45427.0,85678.0,87.41,36.0,H-SC,5.00,155.0,-0.47,0.60,10.35,AR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.06,0.31,106.08,106.72,131245.0,387.0,123723.0,-50.25,39.0,H-SC,3.4,139.0,0.0,0.87,23.77,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.22,-4.37,117.41,107.91,165776.0,-6450.0,141194.0,-22.17,37.0,M-SC,10.47,216.0,-0.04,0.99,0.96,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.36,-9.00,9.91,0.02,15743.0,-15703.0,158856.0,142.19,70.0,M-SC,7.90,234.0,-1.00,1.11,38.18,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.04,-14.14,16.57,0.09,14338.0,-14245.0,86532.0,100.85,50.0,M-SC,3.70,253.0,-0.99,0.61,13.68,OX40N,NTT,DURABLES
67,SIS,528.00,1.40,-22.80,57.07,21.26,48919.0,-25314.0,85718.0,2023.83,53.0,H-SC,3.84,166.0,-0.52,0.60,15.81,OX40N,NTT,MISC
47,KANSAINER,340.00,0.01,-22.89,47.15,13.47,98046.0,-61722.0,207945.0,-68.31,26.0,H-SC,1.29,158.0,-0.63,1.45,5.73,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.75,1.51,69.70,72.27,119888.0,2558.0,172006.0,-12.89,57.0,M-LC,4.45,99.0,0.02,1.20,7.78,XR,NTT,IT
38,INDIAMART,4810.62,-1.06,0.31,106.08,106.72,131245.0,387.0,123723.0,-50.25,39.0,H-SC,3.40,139.0,0.00,0.87,23.77,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.75,1.51,69.70,72.27,119888.0,2558.0,172006.0,-12.89,57.0,M-LC,4.45,99.0,0.02,1.20,7.78,XR,NTT,IT
38,INDIAMART,4810.62,-1.06,0.31,106.08,106.72,131245.0,387.0,123723.0,-50.25,39.0,H-SC,3.40,139.0,0.00,0.87,23.77,AR,ATH,MISC
84,WHIRLPOOL,2270.00,0.64,-3.60,88.71,81.92,98173.0,-4131.0,110667.0,-43.52,38.0,M-SC,3.81,236.0,-0.04,0.77,31.01,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,0.77,-1.33,37.46,35.63,77459.0,-2795.0,206779.0,-15.41,40.0,H-MC,4.17,119.0,-0.04,1.45,15.12,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.22,-4.37,117.41,107.91,165776.0,-6450.0,141194.0,-22.17,37.0,M-SC,10.47,216.0,-0.04,0.99,0.96,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.0,-0.58,-35.46,109.50,35.20,90931.0,-45628.0,83042.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.01,X40,NTT,FOOTWEAR
46,JSWINFRA,342.0,0.79,-9.23,26.95,15.23,49007.0,-18502.0,181845.0,-22.32,27.0,X-MC,6.79,66.0,-0.38,1.27,20.04,X40N,NTT,REALTY
60,RELAXO,1176.0,1.20,-47.55,190.41,52.31,144961.0,-69029.0,76131.0,-44.15,32.0,X-SC,4.17,91.0,-0.48,0.53,1.90,X40N,NTT,FOOTWEAR
27,GLAXO,3466.2,-0.60,-0.79,40.26,39.15,79596.0,-1580.0,197704.0,-20.38,32.0,X-MC,9.77,60.0,-0.02,1.38,27.98,X40,ATH,PHARMA
76,TMPV,600.0,2.71,-14.22,65.75,42.18,155900.0,-39300.0,237110.0,-23.08,32.0,X-LC,1.02,3.0,-0.25,1.66,2.71,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.40,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.38,3.79,X40,NTT,FMCG
76,TMPV,600.0,2.71,-14.22,65.75,42.18,155900.0,-39300.0,237110.0,-23.08,32.0,X-LC,1.02,3.0,-0.25,1.66,2.71,XY24,NTT,AUTO
3,ACC,3906.0,0.01,-21.44,108.96,64.15,203679.0,-51021.0,186930.0,-54.13,54.0,X-SC,1.61,82.0,-0.25,1.31,4.86,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.70,2.21,28.13,30.97,50817.0,3911.0,180649.0,-21.83,53.0,X-MC,1.67,75.0,0.08,1.26,15.05,X40,ATH,INSURANCE
66,SIEMENS,4671.5,0.59,-14.02,45.98,25.51,73568.0,-26095.0,160000.0,-1.82,57.0,H-LC,1.82,49.0,-0.35,1.12,17.91,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.58,-35.46,109.50,35.20,90931.0,-45628.0,83042.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.01,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,0.16,-5.72,33.71,26.07,52041.0,-9360.0,154380.0,-30.55,34.0,X-MC,9.22,55.0,-0.18,1.08,0.16,X40,ATH,APPARELS
55,PGHH,17907.65,-0.33,-4.53,40.10,33.76,76884.0,-9090.0,191730.0,-32.64,33.0,X-MC,3.55,57.0,-0.12,1.34,0.44,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.50,-13.72,72.07,48.45,124672.0,-27519.0,172987.0,-19.87,33.0,X-MC,7.06,64.0,-0.22,1.21,0.50,X40N,ATH,FINANCE
31,HAVELLS,2069.16,0.56,-8.61,44.96,32.48,134769.0,-28237.0,299754.0,-13.20,38.0,X-MC,4.18,67.0,-0.21,2.10,1.83,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.85,-27.73,97.66,42.84,45879.0,-18028.0,46978.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.22,XY24,NTT,MISC
60,RELAXO,1176.00,1.20,-47.55,190.41,52.31,144961.0,-69029.0,76131.0,-44.15,32.0,X-SC,4.17,91.0,-0.48,0.53,1.90,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.58,-35.46,109.50,35.20,90931.0,-45628.0,83042.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.01,X40,NTT,FOOTWEAR
22,DIXON,18931.72,1.46,-6.17,29.57,21.58,38884.0,-8640.0,131499.0,-52.72,34.0,X-MC,6.74,56.0,-0.22,0.92,16.42,X40N,ATH,IT
52,MEDANTA,1486.00,-0.02,4.32,18.94,24.08,25083.0,5486.0,132436.0,-4.04,48.0,X-SC,5.22,89.0,0.22,0.93,24.79,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.34,-14.96,40.26,19.28,118445.0,-51755.0,294201.0,-25.89,59.0,X-LC,7.37,1.0,-0.44,2.06,8.76,X40,ATH,IT
41,INFY,2275.00,0.40,4.61,48.04,54.87,157243.0,14429.0,327317.0,-18.52,57.0,X-LC,2.88,2.0,0.09,2.29,13.28,X40,BTT,IT
76,TMPV,600.00,2.71,-14.22,65.75,42.18,155900.0,-39300.0,237110.0,-23.08,32.0,X-LC,1.02,3.0,-0.25,1.66,2.71,XY24,NTT,AUTO
82,VBL,671.64,0.35,-8.98,49.05,35.67,141010.0,-28359.0,287483.0,-19.83,44.0,X-LC,6.03,4.0,-0.20,2.01,3.63,X40N,ATH,FMCG
43,ITC,452.00,0.40,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.38,3.79,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.20,-12.54,107.73,81.67,88384.0,-11768.0,82042.0,7377.78,53.0,L-SC,19.03,271.0,-0.13,0.57,58.88,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.89,-13.28,60.91,39.54,53435.0,-13431.0,87728.0,-32.23,63.0,M-SC,6.70,220.0,-0.25,0.61,23.70,AR,ATH,AUTO
51,MASFIN,398.61,0.34,-5.48,29.13,22.05,26977.0,-5370.0,92610.0,-18.70,49.0,H-SC,7.39,164.0,-0.20,0.65,34.62,XR,ATH,FINANCE
70,SURYODAY,216.00,0.08,-19.50,52.83,23.03,76158.0,-34914.0,144157.0,57.89,49.0,H-SC,10.10,167.0,-0.46,1.01,43.03,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.36,-9.00,9.91,0.02,15743.0,-15703.0,158856.0,142.19,70.0,M-SC,7.90,234.0,-1.00,1.11,38.18,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.36,-9.00,9.91,0.02,15743.0,-15703.0,158856.0,142.19,70.0,M-SC,7.90,234.0,-1.00,1.11,38.18,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.25,0.77,20.02,20.95,50792.0,1936.0,253704.0,-0.21,76.0,X-LC,13.96,31.0,0.04,1.78,36.68,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,1.17,-37.26,111.83,32.89,54166.0,-28770.0,48436.0,-698.49,63.0,L-MC,5.58,259.0,-0.53,0.34,33.39,XR,NTT,BANKS
0,5PAISA,593.00,2.37,-34.99,73.67,12.90,104643.0,-76453.0,142043.0,109.13,58.0,H-SC,14.00,161.0,-0.73,0.99,18.11,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,0.31,5.40,14.58,20.77,35981.0,12642.0,246786.0,-3.83,60.0,X-MC,3.68,62.0,0.35,1.73,12.69,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,44.60
2,50,75.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.74
MC    29.19
LC    25.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.98
X40      21.14
X40N     11.88
XR        9.65
XY25      9.06
AR        9.00
OX40N     7.69
X200      1.85
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.03
X-MC    22.76
X-LC    19.95
M-SC    12.05
X-SC     8.25
H-MC     4.72
H-LC     2.97
L-SC     1.41
M-MC     1.37
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.76,-5.11,39.11
IT,13.70,-15.40,74.97
FINANCE,9.47,-14.41,62.76
MISC,7.46,-22.13,72.17
PAINTS,5.93,-8.84,25.01
ELECTRICAL,5.90,-11.24,51.99
INSURANCE,4.45,0.55,34.49
PHARMA,3.88,-0.71,32.94
AUTO,3.35,-19.93,71.36


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3040388.0,21
AR,1286599.0,10
XR,1282112.0,13
X40,1017495.0,14
X40N,750183.0,9
OX40N,692406.0,10
XY25,346973.0,6
SR,279377.0,2
MH,72380.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3544515.0,25
M-SC,1337213.0,15
X-MC,1229832.0,16
X-LC,912539.0,11
X-SC,741408.0,8
H-MC,404667.0,3
L-SC,259674.0,3
H-LC,133178.0,2
M-LC,119888.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1208102.0      6
           AR         875977.0      5
           XR         788107.0      7
M-SC       XY24       781589.0      6
X-MC       X40        443718.0      7
X-LC       X40        399388.0      5
X-MC       XY24       357860.0      3
H-SC       OX40N      320572.0      4
X-SC       X40N       292378.0      3
M-SC       OX40N      284663.0      5
H-SC       SR         279377.0      2
X-SC       XY24       274641.0      3
X-MC       X40N       248544.0      4
X-LC       XY24       227070.0      2
H-MC       AR         213541.0      2
X-LC       X40N       209261.0      2
H-MC       XY24       191126.0      1
X-MC       XY25       179710.0      2
X-SC       X40        174389.0      2
L-SC       XR         172503.0      2
M-SC       XR         147448.0      2
           AR         123513.0      2
M-LC       XR         119888.0      1
L-SC       OX40N       87171.0      1
X-LC       XY25        76820.0      2
H-LC       AR          73568.0      1
H-SC       MH          72380.0      1
H-LC       X200        59610.0      1
L-MC       XR          54166.0      1
M-MC       XY25        49247.0      1
L-LC       XY25        41196.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
